In [44]:
import pandas as pd
import gensim
from gensim.models import Word2Vec
from gensim.models import KeyedVectors 
import time
list_csv = ['data/attraction_places.csv',
           'data/hotel_places.csv',
           'data/restaurant_places.csv']
attraction_e2 =  KeyedVectors.load('model/attraction_tag_e2.model')
attraction_e3 =  KeyedVectors.load('model/attraction_tag_e3.model')
attraction_e4 =  KeyedVectors.load('model/attraction_tag_e4.model')
attraction_e5 =  KeyedVectors.load('model/attraction_tag_e5.model')

restaurant_e5 =  KeyedVectors.load('model/restaurant_tag_e5.model')
attraction_user =  KeyedVectors.load('model/attraction_user_e2.model')
#model2 =  Word2Vec().wv.load('model/eng_hotel_tag.model')
#model3 =  Word2Vec().wv.load('model/restaurant_tag.model')

In [2]:
#entities_list = ['324890', '3808315', '590748']
entities_list = ['10183192', '10020510', '10130671', '10020529', '10020530']

In [3]:
def rank(srcwv, entry, positive=None, negative=None):
    subwv = srcwv[entry]
    newwv = Word2Vec(size=len(subwv[0]))
    # add entries to new vector
    for i in range(len(entry)):
        newwv.wv.add(entry[i], subwv[i])
    # add p, n to new vector
    for w in positive:
        newwv.wv.add(w, srcwv[w])
    for w in negative:
        newwv.wv.add(w, srcwv[w])
    rank = newwv.most_similar(positive=positive,
                              negative=negative,
                              topn=len(entry))
    return rank

In [4]:
#rank(attraction_e5, entities_list, positive=['320359'], negative=['10020510'])
rank(attraction_e5, entities_list, positive=['10677413'], negative=[])

/home/jahn/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:14: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  
/home/jahn/anaconda3/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('10183192', 0.48084136843681335),
 ('10020510', 0.30906859040260315),
 ('10130671', 0.30425506830215454),
 ('10020529', 0.2980351448059082),
 ('10020530', 0.1685188263654709)]

In [55]:
df_place = pd.read_csv(list_csv[2])
df_place

,class,placeId,score,name_kor,name_eng,location
0,restaurants,10000530,4.666667,닭다리칼국수,Dakdari Kalguksu,서울 성동구 연무장3길 21
1,restaurants,10000553,4.500000,동해도 을지로점,Donghaedo Euljiro,서울 중구 남대문로10길 9
2,restaurants,10000564,4.500000,명장도가니,Myeongjang Dogani,서울 노원구 석계로18길 15
3,restaurants,10000599,3.560976,빌즈 광화문,bills Gwanghwamun,"37.5717,126.97971"
4,restaurants,10000653,4.000000,벽돌집,Byeokdopjip,서울 마포구 와우산로21길 21
5,restaurants,10025748,4.000000,뱀부하우스,Bamboo House,"37.507,127.03664"
6,restaurants,10040756,3.818182,117 파크,117 Park,"37.571552,126.970795"
7,restaurants,10058097,5.000000,Nogari,Nogari,"서울 153-4 Insa-dong, Jongno-gu, Seoul, South Korea"
8,restaurants,10058560,4.333333,사마르칸트시티,Samarkand City,서울 중구 을지로42길 16
9,restaurants,10060003,5.000000,와인나라,Wine Nara,서울 서초구 서래로 8길 1


In [56]:
#model = KeyedVectors.load('model/attraction_user_e5.model')
#model = attraction_e2
model = restaurant_e5
df_place = pd.read_csv(list_csv[2])
p = ['9979865']
n = []
#n = ['1163226','14112185','1537022','1552278']
print('positive: ', [df_place[df_place['placeId']==int(i)].name_kor.tolist()[0]
                     for i in p])
print('negative: ', [df_place[df_place['placeId']==int(i)].name_kor.tolist()[0]
                     for i in n])
print('상위 10개 유사도')
top10 = model.most_similar(positive=p, negative=n, topn=10)
cnt = 0
#print(top10)
result = pd.DataFrame(columns=['name', 'similarity'])
for place in top10:
    name = df_place[df_place['placeId'] == int(place[0])].name_kor.tolist()
    if pd.isna(name):
        name = df_place[df_place['placeId'] == int(place[0])].name_eng.tolist()
    result.loc[cnt] = [name,place[1]]
    cnt+=1
result

positive:  ['홍대개미']
negative:  []
상위 10개 유사도


/home/jahn/anaconda3/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


,name,similarity
0,[랑빠스81],0.785278
1,[홍대 조폭 떡볶이],0.767571
2,[할머니의 레시피],0.758972
3,[안남],0.747974
4,[육지],0.747566
5,[르스타일],0.745619
6,[온더보더 IFC몰점],0.741591
7,[모녀꼬마마약김밥],0.739198
8,[더그리핀바],0.737045
9,[동경암],0.736735


In [43]:
#유사도 행렬 추출
index = gensim.similarities.MatrixSimilarity(gensim.matutils.Dense2Corpus(model.syn0))

/home/jahn/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).
  
/home/jahn/anaconda3/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [30]:
model.wv.syn0[model.wv.vocab['2194168'].index]

/home/jahn/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.
/home/jahn/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).
  """Entry point for launching an IPython kernel.


array([ 0.13839234, -0.17565385, -0.04056159,  0.05154056,  0.071432  ,
        0.08407546, -0.1912874 ,  0.37984294,  0.14532113, -0.01730778,
        0.03792423, -0.07237744,  0.1877766 ,  0.0317835 , -0.16698128,
        0.02196448,  0.02686762, -0.21212645, -0.07353383,  0.08389873,
        0.1590257 ,  0.1513149 ,  0.15425241, -0.2761336 ,  0.07657761,
       -0.03714213, -0.09613925, -0.04326359, -0.06178857,  0.08635211,
       -0.0091673 , -0.06388297,  0.01732828,  0.15332611,  0.08311914,
       -0.05673724, -0.11668353, -0.00715934,  0.11011361,  0.21735202,
       -0.03765745,  0.1038878 ,  0.12084716,  0.17029604,  0.04210516,
        0.09322475, -0.03111278, -0.01132023, -0.02423344,  0.19384491,
        0.05603138, -0.06513225, -0.11386864,  0.17850171, -0.10816479,
        0.2787438 , -0.02870308,  0.01637081,  0.10937627, -0.20033632,
        0.16194044, -0.03286066, -0.01941763, -0.06528051, -0.27314603,
       -0.13537593, -0.06400071,  0.17042507,  0.10177206, -0.12

In [31]:
model.wv.syn0[model.wv.vocab['324888'].index]

/home/jahn/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.
/home/jahn/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).
  """Entry point for launching an IPython kernel.


array([ 7.19788894e-02,  1.95781499e-01,  1.44076884e-01, -1.35519594e-01,
       -8.97339210e-02,  1.34750307e-02, -1.02432355e-01,  3.38946849e-01,
        2.68747121e-01,  2.13579878e-01, -4.69534472e-02, -4.36314464e-01,
        1.02688156e-01, -6.86596846e-03,  2.02636212e-01, -5.31446561e-02,
        5.27765928e-03, -6.49338365e-02, -1.89654142e-01,  3.57595906e-02,
       -7.73570985e-02,  1.13218620e-01,  1.00510977e-02,  3.34134349e-03,
       -2.83511709e-02,  1.94658518e-01,  3.87370996e-02, -5.42372372e-03,
        2.99796939e-01, -1.14560522e-01,  9.47400331e-02,  6.54617399e-02,
       -6.81081340e-02, -1.33168057e-01,  1.37904808e-01,  1.36948898e-01,
        6.36152166e-04,  1.20404668e-01,  3.62119712e-02,  6.73462301e-02,
       -9.60579589e-02, -1.38206407e-01, -1.37796417e-01, -8.22406933e-02,
        1.56551927e-01,  1.50650635e-01, -1.51445314e-01, -1.33228526e-01,
        1.02156296e-01, -3.55602875e-02,  3.53231728e-02,  6.84826300e-02,
       -6.88300356e-02,  

In [29]:
model.index2word

['2194168',
 '324888',
 '553546',
 '1169465',
 '592506',
 '554537',
 '1379963',
 '320359',
 '1046419',
 '324891',
 '325043',
 '554582',
 '324907',
 '1958940',
 '1552278',
 '6671988',
 '2203080',
 '592486',
 '554528',
 '1842046',
 '324887',
 '1235392',
 '1862191',
 '324890',
 '1519813',
 '609340',
 '592495',
 '1551271',
 '3322253',
 '6847900',
 '598499',
 '1957627',
 '592658',
 '590748',
 '1604009',
 '1554603',
 '4798715',
 '554303',
 '2571660',
 '2008729',
 '8842556',
 '554634',
 '6656879',
 '592490',
 '4798711',
 '1579821',
 '1766551',
 '1962594',
 '3600495',
 '4084344',
 '1478599',
 '8737878',
 '1440196',
 '1537022',
 '6570666',
 '1643534',
 '2094166',
 '8693245',
 '554533',
 '3694519',
 '554551',
 '3808315',
 '592484',
 '3568803',
 '4095685',
 '4094861',
 '1990451',
 '3871936',
 '3822061',
 '2011401',
 '1960138',
 '561808',
 '6378743',
 '7154750',
 '553246',
 '1418675',
 '1809989',
 '2341905',
 '1836198',
 '1831099',
 '1758805',
 '1675961',
 '1958930',
 '553213',
 '3901164',
 '13833

In [26]:
kv = KeyedVectors(300)

In [ ]:
kv.